In [ ]:
# WordSegment 라이브러리 설치
!pip install wordsegment

In [ ]:
import pandas as pd
import nltk
from collections import defaultdict, Counter  # defaultdict 추가
from nltk.tokenize import word_tokenize
from wordsegment import load, segment

# 필요한 데이터 로드 (nltk punkt)
nltk.download('punkt')

# WordSegment 데이터 로드
load()

In [ ]:
# CSV 파일 불러오기
from google.colab import files
uploaded = files.upload()
csv_file_path = list(uploaded.keys())[0]
df = pd.read_csv(csv_file_path)

In [ ]:
# 단어 빈도 및 해당 단어를 포함하는 ID 목록 계산 함수
def calculate_word_frequency_with_ids(ids):
    word_freq = Counter()
    word_id_map = defaultdict(set)  # defaultdict를 set으로 변경하여 중복 제거

    for id_ in ids:
        words = word_tokenize(id_)  # ID를 단어 토큰으로 나눔
        for word in words:
            if word.isalpha():  # 알파벳으로만 이루어진 경우
                # 붙어있는 단어를 분리
                segmented_words = segment(word.lower())
                word_freq.update(segmented_words)
                for segmented_word in segmented_words:
                    word_id_map[segmented_word].add(id_)  # set에 추가하여 중복 제거

    # ID 목록을 다시 리스트로 변환
    for word in word_id_map:
        word_id_map[word] = list(word_id_map[word])

    return word_freq, word_id_map

In [ ]:

# ID 컬럼에서 단어 빈도 및 ID 목록 계산
ids = df['ID'].astype(str).tolist()  # 'ID' 컬럼 이름을 실제 컬럼 이름으로 변경
word_frequencies, word_id_map = calculate_word_frequency_with_ids(ids)

# 단어 빈도 데이터를 DataFrame으로 변환
word_freq_df = pd.DataFrame(word_frequencies.items(), columns=['Word', 'Frequency'])

# 해당 단어를 포함하는 ID 목록을 추가
word_freq_df['IDs'] = word_freq_df['Word'].map(word_id_map)


In [ ]:
# 빈도가 높은 순서로 정렬
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False)

# 상위 10개의 빈도 높은 단어 및 해당 단어가 포함된 ID 출력
print(word_freq_df.head(10))

# CSV 파일로 저장
output_csv_file_path = "word_frequencies_with_ids_output.csv"
word_freq_df.to_csv(output_csv_file_path, index=False)

print(f"Download Complete!": {output_csv_file_path}")